# Module 6: Update your app

### Objectives

* Perform a rolling update using kubectl.

### Updating an application

Users expect applications to be available all the time and developers are expected to deploy new versions of them several times a day. In Kubernetes this is done with rolling updates. **Rolling updates** allow Deployments' update to take place with zero downtime by incrementally updating Pods instances with new ones. The new Pods will be scheduled on Nodes with available resources.

In the previous module we scaled our application to run multiple instances. This is a requirement for performing updates without affecting application availability. By default, the maximum number of Pods that can be unavailable during the update and the maximum number of new Pods that can be created, is one. Both options can be configured to either numbers or percentages (of Pods). In Kubernetes, updates are versioned and any Deployment update can be reverted to a previous (stable) version.

> Rolling updates allow Deployments' update to take place with zero downtime by incrementally updating Pods instances with new ones.

## Rolling updates overview

<div>
<img src="https://d33wubrfki0l68.cloudfront.net/30f75140a581110443397192d70a4cdb37df7bfc/fa906/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates1.svg" width=400>

<img src="https://d33wubrfki0l68.cloudfront.net/678bcc3281bfcc588e87c73ffdc73c7a8380aca9/703a2/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates2.svg" width=400>
</div>

<div>
<img src="https://d33wubrfki0l68.cloudfront.net/9b57c000ea41aca21842da9e1d596cf22f1b9561/91786/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates3.svg" width=400>

<img src="https://d33wubrfki0l68.cloudfront.net/6d8bc1ebb4dc67051242bc828d3ae849dbeedb93/fbfa8/docs/tutorials/kubernetes-basics/public/images/module_06_rollingupdates4.svg" width=400>
 
</div>
    


Similar to application Scaling, if a Deployment is exposed publicly, the Service will load-balance the traffic only to available Pods during the update. An available Pod is an instance that is available to the users of the application.

Rolling updates allow the following actions:

* Promote an application from one environment to another (via container image updates)
* Rollback to previous versions
* Continuous Integration and Continuous Delivery of applications with zero downtime

> If a Deployment is exposed publicly, the Service will load-balance the traffic only to available Pods during the update.



In the following lab we'll update our application to a new version, and also perform a rollback.

### Lab 6 - Update your app
#### Step 1 of 3: Update the version of the app

To list your deployments use the get deployments command: `kubectl get deployments`


In [ ]:
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
POD_NAME=$(kubectl get pods -o go-template --template '{% raw %}{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}{% endraw %}' | grep $studentId | head -n 1) 
port=$(expr $(echo ${studentId} | tr -cd '[[:digit:]]') + 8000)

kubectl get deployments | grep $studentId


To list the running Pods use the `get pods` command:

In [ ]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

To view the current image version of the app, run a `describe` command against the deployments (look at the Image field):

In [ ]:
kubectl describe deployments/kubernetes-bootcamp-$studentId

To update the image of the application to version 2, use the `set image` command, followed by the deployment name and the new image version:

In [ ]:
kubectl set image deployments/kubernetes-bootcamp-$studentId kubernetes-bootcamp=jocatalin/kubernetes-bootcamp:v2

The command notified the Deployment to use a different image for your app and initiated a rolling update. Check the status of the new Pods, and view the old one terminating with the `get pods` command:

> To execute a multiple times, use CTRL-Enter while positioned on that cell.

In [ ]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

#### Step 2 of 3: Verify an update

First, let’s check that the App is running. To find out the exposed IP and Port we can use `describe service`:

In [ ]:
kubectl describe deployments/kubernetes-bootcamp-$studentId

Create an environment variable called NODE_PORT that has the value of the Node port assigned.
And another one that compute NODE_NAME from a  `kubectl describe pods ` command. 

> Before we do so, let's recreate the service we deleted at the end of previous module.

In [ ]:
kubectl expose deployment/kubernetes-bootcamp-$studentId --type="NodePort" --port 8080

In [ ]:
NODE_PORT=$(kubectl get services/kubernetes-bootcamp-$studentId -o go-template='{% raw %}{{(index .spec.ports 0).nodePort}}{% endraw %}')
echo NODE_PORT=$NODE_PORT
POD_NAME=$(kubectl get pods -o go-template --template '{% raw %}{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}{% endraw %}' | grep $studentId | head -n 1) 
NODE_NAME=$(kubectl describe pods $POD_NAME | grep Node: | head -n 1)
NODE_NAME=${NODE_NAME#Node:} 
NODE_NAME=${NODE_NAME%/*} 
NODE_NAME=$(echo $NODE_NAME | xargs)
echo NODE_NAME=$NODE_NAME


Next, we’ll do a `curl` to the the exposed IP and port:

In [ ]:
curl http://$NODE_NAME:$NODE_PORT

We hit a different Pod with every request and we see that all Pods are running the latest version (v2).

The update can be confirmed also by running a rollout status command:

**This command might hang, please use the stop button to stop it before you move on.**



In [ ]:
kubectl rollout status deployments/kubernetes-bootcamp-$studentId

To view the current image version of the app, run a describe command against the Pods:

In [ ]:
kubectl describe pods -l app=kubernetes-bootcamp-$studentId

We run now version 2 of the app (look at the Image field)

#### Step 3 of 3: Rollback an update

Let’s perform another update, and deploy image tagged as `v10` :

In [ ]:
kubectl set image deployments/kubernetes-bootcamp-$studentId kubernetes-bootcamp=gcr.io/google-samples/kubernetes-bootcamp:v10

Use `get deployments` to see the status of the deployment:

In [ ]:
kubectl get deployment | grep $studentId

And something is wrong… We do not have the desired number of Pods available. List the Pods again:

In [ ]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

A `describe` command on the Pods should give more insights:

In [ ]:
kubectl describe pods -l app=kubernetes-bootcamp-$studentId

There is no image called `v10` in the repository. Let’s roll back to our previously working version. We’ll use the `rollout` undo command:

In [ ]:
kubectl rollout undo deployments/kubernetes-bootcamp-$studentId

The `rollout` command reverted the deployment to the previous known state (v2 of the image). Updates are versioned and you can revert to any previously know state of a Deployment. List again the Pods:

> To execute a multiple times, use CTRL-Enter while positioned on that cell.

In [ ]:
kubectl get pods -l app=kubernetes-bootcamp-$studentId

Four Pods are running. Check again the image deployed on the them:

In [ ]:
kubectl describe pods -l app=kubernetes-bootcamp-$studentId

We see that the deployment is using a stable version of the app (v2). The Rollback was successful.

This terminates module 6 and the workshop, let's perform some cleanup.

In [ ]:
kubectl delete services/kubernetes-bootcamp-$studentId
kubectl delete deployment/kubernetes-bootcamp-$studentId


[Continue to conclusion](./8-WKSHP-Conclusion.ipynb) or [return to main Notebook](./1-WKSHP-K8S101.ipynb).